In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
df = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(),annot=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.pairplot(df)

In [ ]:
sns.distplot(x=df.Glucose,hist=True)

In [ ]:
def age_group(Age):
    if Age in range(21,31):
        return "age_group_1"
         
    elif Age in range(31,41):
        return "age_group_2"
        
    elif Age in range(41,51):
        return "age_group_3"
         
    elif Age in range(51,61):
        return "age_group_4"
        
    else:
        return "age_group_5"
        

In [ ]:
df['age_groups'] = df.Age.apply(age_group)

In [ ]:
df

In [ ]:
df.age_groups.value_counts()

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(x='age_groups',data=df,hue='Outcome')

In [ ]:
X = df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']].values
y = df.Outcome.values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#### creating tensors
### it is a must thatyour independent features be converted into FLOAT TENSORS
## this is not that much importent for your dependent feature
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
df.shape

In [ ]:
## creating model with Pytorch

class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden_1=20,hidden_2=20,output_features=2):
        super().__init__()
        self.f_connected_1 = nn.Linear(input_features,hidden_1)
        self.f_connected_2 = nn.Linear(hidden_1,hidden_2)
        self.out = nn.Linear(hidden_2,output_features)
    def forward(self,x):
        x = F.relu(self.f_connected_1(x))
        x = F.relu(self.f_connected_2(x))
        x = self.out(x)
        return x

In [ ]:
#### instantiate my ANN MODEL

torch.manual_seed(20)
model = ANN_Model()

In [ ]:
model.parameters

In [ ]:
### backward propagation-- define the Loss_function, define the optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
epochs=500
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print('epoch number: {} and the loss : {}'.format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
### plot the loss function
plt.plot(range(epochs),final_losses)
plt.ylabel('loss')
plt.xlabel('epoch')

In [ ]:
### prediction in x_test data
predictions=[]
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predictions)
cm

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual values')
plt.ylabel('predicted values')

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,predictions)

In [ ]:
score

In [ ]:
list(df.iloc[0,:-2])

In [ ]:
## new data
lst1 = [6.0, 130.0, 72.0, 40.0, 0.0, 25.6, 0.627, 45.0]

In [ ]:
new_data = torch.FloatTensor(lst1)

In [ ]:
## predict new data using new data
with torch.no_grad():
    print(model(new_data))
    print(model(new_data).argmax().item())